In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import LSTM, Convolution1D, Flatten, Dropout, MaxPooling1D, Dense, Merge
# from keras.layers.convolution import 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import keras as kr
from keras.callbacks import ModelCheckpoint
from sklearn.utils import class_weight
from keras.optimizers import SGD,adam
from keras.regularizers import l2

from sklearn.preprocessing import LabelEncoder
kr.__version__

Using TensorFlow backend.


'2.0.4'

In [2]:
base_dir_tr='training/'
base_dir_vl='validation/'
weight_log='weight_logs/'

In [3]:
top_words = 20000
dtr_1=pd.read_csv(base_dir_tr+'data_manipulation_2_2_train.csv',encoding='ISO-8859-1')
dtv_1=pd.read_csv(base_dir_vl+'data_manipulation_2_2_valid.csv',encoding='ISO-8859-1')
dtr_2=pd.read_csv(base_dir_tr+'data_cat_tr.csv',encoding='ISO-8859-1')
dtv_2=pd.read_csv(base_dir_vl+'data_cat_vl.csv',encoding='ISO-8859-1')

In [4]:
#### category part
dcat_l=[]
for cc in range(3):
    dum=pd.get_dummies(dtr_2[dtr_2.columns[cc+1]])
    col_i=[dtr_2.columns[cc+1]+'_'+str(i) for i in dum.columns]
    dum.columns=col_i
    dcat_l.append(dum)
dcat_tr_1=pd.concat(dcat_l,axis=1)
Xcat_tr=dcat_tr_1.as_matrix()

dcat_l=[]
for cc in range(3):
    dum=pd.get_dummies(dtv_2[dtv_2.columns[cc+1]])
    col_i=[dtv_2.columns[cc+1]+'_'+str(i) for i in dum.columns]
    dum.columns=col_i
    dcat_l.append(dum)
dcat_tv_1=pd.concat(dcat_l,axis=1)
Xcat_tv=dcat_tv_1.as_matrix()

print(len(dcat_tr_1),len(dcat_tv_1),len(dcat_tv_1.columns))

36283 11838 251


In [5]:
Xsd=dtr_1.rank_short_des.as_matrix()
Xsd=np.array([ast.literal_eval(cd) for cd in Xsd])

Xti=dtr_1.rank_title.as_matrix()
Xti=np.array([ast.literal_eval(cd) for cd in Xti])

yco=dtr_1.conci

In [6]:
Xsd_v=dtv_1.rank_short_des.as_matrix()
Xsd_v=np.array([ast.literal_eval(cd) for cd in Xsd_v])

Xti_v=dtv_1.rank_title.as_matrix()
Xti_v=np.array([ast.literal_eval(cd) for cd in Xti_v])

In [7]:
# L_Xsd=[len(x) for x in Xsd]
# L_Xti=[len(x) for x in Xti]

# dLsd=pd.DataFrame(L_Xsd)
# dLti=pd.DataFrame(L_Xti)

# dLsd[dLsd<200].hist(bins=50)

In [8]:
max_review_length_s = 150
max_review_length_t = 40
Xsd = sequence.pad_sequences(Xsd, maxlen=max_review_length_s)
Xti = sequence.pad_sequences(Xti, maxlen=max_review_length_t)

Xsd_v = sequence.pad_sequences(Xsd_v, maxlen=max_review_length_s)
Xti_v = sequence.pad_sequences(Xti_v, maxlen=max_review_length_t)

In [9]:
X=[]
for i,j,k in zip(Xsd,Xti,Xcat_tr):
    X.append(list(k)+list(j)+list(i))
X=np.array(X)

X_v=[]
for i,j,k in zip(Xsd_v,Xti_v,Xcat_tv):
    X_v.append(list(k)+list(j)+list(i))
X_v=np.array(X_v)

# np.savetxt(base_dir_vl+'data_input_yco_3_1.txt',X_v)
np.savetxt(base_dir_vl+'data_input_yco_4_2.txt',X_v)

In [10]:
len(X[0][251:])==len(np.array(list(Xti[0])+list(Xsd[0])))

True

In [11]:
len(X[0][:251])==len(np.array(list(Xcat_tr[0])))

True

#### yco model

In [12]:
# model.save('model_yco_1_2_3.h5') OKE (74.05%)
yco.value_counts()/yco.value_counts().sum()

1    0.685335
0    0.314665
Name: conci, dtype: float64

In [13]:
X_train, X_test, yco_train, yco_test = train_test_split(X, yco, test_size=0.3, random_state=21)

X_train_nl=np.array([x[251:] for x in X_train])
X_train_ct=np.array([x[:251] for x in X_train])

X_test_nl=np.array([x[251:] for x in X_test])
X_test_ct=np.array([x[:251] for x in X_test])

In [14]:
yco_test.value_counts()/yco_test.value_counts().sum()

1    0.686082
0    0.313918
Name: conci, dtype: float64

In [15]:
# len(X_train_nl)
X_train

array([[    0,     0,     0, ...,    50,   135,  5013],
       [    0,     0,     0, ...,   173,     6,   250],
       [    0,     0,     0, ...,  4831,    21,   641],
       ..., 
       [    0,     0,     0, ...,  1535,   233,   117],
       [    0,     0,     0, ...,   962,    18, 14122],
       [    1,     0,     0, ...,  2551,  3305,     0]], dtype=int32)

In [16]:
X_train_nl

array([[    0,     0,     0, ...,    50,   135,  5013],
       [    0,     0,     0, ...,   173,     6,   250],
       [    0,     0,     0, ...,  4831,    21,   641],
       ..., 
       [    0,     0,     0, ...,  1535,   233,   117],
       [    0,     0,     0, ...,   962,    18, 14122],
       [    0,     0,     0, ...,  2551,  3305,     0]], dtype=int32)

In [21]:
np.random.seed(21)
#### Creating the model_1
embedding_vecor_length = 150
model_nl = Sequential()
model_nl.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length_s+max_review_length_t))
model_nl.add(Convolution1D(128, 3, border_mode='same'))
model_nl.add(Convolution1D(32, 3, border_mode='same'))
# model_nl.add(LSTM(30))
model_nl.add(Flatten())
model_nl.add(Dropout(0.1))
model_nl.add(Dense(100,activation='relu', W_regularizer = l2(.01)))
# model_nl.add(Dense(100,activation='relu', W_regularizer = l2(.01)))
model_nl.add(Dropout(0.1))
model_nl.add(Dense(3,activation='relu'))


# W_regularizer = l2(.01)
#### Creating the model_2
model_ct = Sequential()
model_ct.add(Dense(50, input_shape=(251,),activation='relu'))
model_ct.add(Dense(50, input_shape=(251,),activation='relu'))
model_ct.add(Dense(50, input_shape=(251,),activation='relu'))
model_ct.add(Dense(25, input_shape=(251,),activation='relu'))
model_ct.add(Dense(25, input_shape=(251,),activation='relu'))
model_ct.add(Dense(12, input_shape=(251,),activation='relu'))
model_ct.add(Dropout(0.1))
model_ct.add(Dense(2,activation='relu'))


model = Sequential()
model.add(Merge([model_ct,model_nl], mode='concat'))
model.add(Dense(1,activation='sigmoid'))

model2=model

# opti = SGD(lr=0.01)
opti = adam(lr=0.00001) #0.001
# model.compile(loss = "categorical_crossentropy", optimizer = opt)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])
model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy','mse'])
# model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','mse'])


# checkpoint
# filepath=weight_log+"weights_best_ycl_3_1.hdf5"
# filepath=weight_log+"weights_best_yco_4_1.hdf5"
filepath=weight_log+"weights_best_yco_4_2.hdf5"


checkpoint = ModelCheckpoint(filepath, monitor='val_mean_squared_error', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`
/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, kernel_regularizer=<keras.reg..., activation="relu")`
/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:31: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [22]:
model.input_shape

[(None, 251), (None, 190)]

In [23]:
print(X_train_ct.shape,X_train_nl.shape)

(25398, 251) (25398, 190)


In [24]:
model.fit([X_train_ct,X_train_nl],yco_train, validation_data=([X_test_ct,X_test_nl],yco_test),callbacks=callbacks_list, epochs=100, batch_size=100, verbose=1)

Train on 25398 samples, validate on 10885 samples
Epoch 1/100
25398/25398 [==============================] - 36s - loss: 2.4042 - acc: 0.6844 - mean_squared_error: 0.2317 - val_loss: 2.1618 - val_acc: 0.6861 - val_mean_squared_error: 0.2119
Epoch 2/100
25398/25398 [==============================] - 62s - loss: 1.9868 - acc: 0.6850 - mean_squared_error: 0.2095 - val_loss: 1.8167 - val_acc: 0.6861 - val_mean_squared_error: 0.2033
Epoch 3/100
25398/25398 [==============================] - 61s - loss: 1.6683 - acc: 0.6929 - mean_squared_error: 0.1935 - val_loss: 1.5213 - val_acc: 0.7306 - val_mean_squared_error: 0.1796
Epoch 4/100
25398/25398 [==============================] - 63s - loss: 1.4265 - acc: 0.7435 - mean_squared_error: 0.1788 - val_loss: 1.3259 - val_acc: 0.7610 - val_mean_squared_error: 0.1720
Epoch 5/100
25398/25398 [==============================] - 63s - loss: 1.2566 - acc: 0.7660 - mean_squared_error: 0.1718 - val_loss: 1.1822 - val_acc: 0.7558 - val_mean_squared_error: 0.

KeyboardInterrupt: 

In [25]:
# model2.load_weights(weight_log+"weights_best_yco_3_1.hdf5")
# model2.load_weights(weight_log+"weights_best_yco_4_1.hdf5")
model2.load_weights(weight_log+"weights_best_yco_4_2.hdf5")

model2.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy','mse'])

In [26]:
#### Evaluation on the test set
yco_predc=model2.predict_classes([X_test_ct,X_test_nl])
yco_predp=model2.predict([X_test_ct,X_test_nl])

yco_predc=[i[0] for i in yco_predc]
yco_predp=[i[0] for i in yco_predp]

scores = model2.evaluate([X_test_ct,X_test_nl], yco_test, verbose=0)
mse_c=mse(yco_test,yco_predc)
mse_p=mse(yco_test,yco_predp)
print("Accuracy: %.2f%%" % (scores[1]*100)) 
print("MSE_class : ",mse_c)
print("MSE_prob : ",mse_p)

10880/10885 [============================>.] - ETA: 0sAccuracy: 82.82%
MSE_class :  0.17179604961
MSE_prob :  0.127724313221


In [27]:
# model2.save('model_yco_3_3.h5') # MSE prob 82.09% 0.1310365
# model2.save('model_yco_3_4.h5') # 82.35%     0.13095322

# model_nl.save('model_yco_4_1(1).h5') #0.130033
# model_ct.save('model_yco_4_1(2).h5')

model_nl.save('model_yco_4_2(1).h5') #0.127724
model_ct.save('model_yco_4_2(2).h5')

### Validation

In [28]:
X_vl=np.loadtxt(base_dir_vl+'data_input_yco_4_2.txt')

yco_model_1=load_model('model_yco_4_2(1).h5')
yco_model_2=load_model('model_yco_4_2(2).h5')
yco_model = Sequential()
yco_model.add(Merge([yco_model_2,yco_model_1], mode='concat'))
yco_model.add(Dense(1,activation='sigmoid'))

yco_model.load_weights(weight_log+"weights_best_yco_4_2.hdf5")
yco_model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy','mse'])

/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/keras/models.py:248: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [29]:
X_vl_nl=np.array([x[251:] for x in X_vl])
X_vl_ct=np.array([x[:251] for x in X_vl])

In [30]:
yco_predp=yco_model.predict([X_vl_ct,X_vl_nl])
yco_predp=[i[0] for i in yco_predp]

In [31]:
with open('conciseness_valid.predict','w') as fcl:
    for pp in yco_predp:
        fcl.write(str(pp))
        fcl.write('\n')
fcl.close()